/tmp/ipykernel_13082/2023831833.py:5: DeprecationWarning: the ``ned`` module has been moved to astroquery.ipac.ned, please update your imports.
  from astroquery.ned import Ned


In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.simbad import Simbad
from astroquery.ned import Ned
from astroquery.sdss import SDSS

# -------------------------
# SIMBAD query 
# -------------------------
Simbad.add_votable_fields('otype')

def query_simbad_improved(ra, dec, radius_arcsec=15):
    """
    Query SIMBAD by coordinates and return first match.
    Returns (name, ra_deg, dec_deg) or (None, None, None)
    """
    try:
        pos = SkyCoord(ra*u.deg, dec*u.deg)
        result = Simbad.query_region(pos, radius=radius_arcsec*u.arcsec)
        if result is None or len(result) == 0:
            return None, None, None

        row = result[0]
        name = row['MAIN_ID']
        if isinstance(name, bytes):
            name = name.decode('utf-8')

        # RA/DEC conversion: SIMBAD returns RA in hours by default
        ra_deg = SkyCoord(row['RA'], row['DEC'], unit=(u.hourangle, u.deg)).ra.deg
        dec_deg = SkyCoord(row['RA'], row['DEC'], unit=(u.hourangle, u.deg)).dec.deg

        return name, ra_deg, dec_deg

    except Exception:
        return None, None, None

# -------------------------
# NED query
# -------------------------
def query_ned(ra, dec, radius_arcsec=10):
    try:
        pos = SkyCoord(ra*u.deg, dec*u.deg)
        result = Ned.query_region(pos, radius=radius_arcsec*u.arcsec)
        if result is None or len(result) == 0:
            return None, None, None

        row = result[0]
        name = row['Object Name']
        ra_out = row['RA']
        dec_out = row['DEC']
        return name, ra_out, dec_out

    except Exception:
        return None, None, None

# -------------------------
# SDSS query
# -------------------------
def query_sdss(ra, dec, radius_arcsec=10):
    try:
        pos = SkyCoord(ra*u.deg, dec*u.deg)
        xid = SDSS.query_region(pos, radius=radius_arcsec*u.arcsec, spectro=True)
        if xid is None or len(xid) == 0:
            return None, None, None

        row = xid[0]
        name = f"SDSS J{row['ra']:.5f}{row['dec']:+.5f}"
        return name, row['ra'], row['dec']

    except Exception:
        return None, None, None


# -------------------------
# Process CSV catalog
# -------------------------
def process_galaxy_catalog(csvfile, simbad_radius=10, ned_radius=10, sdss_radius=10):
    df = pd.read_csv(csvfile)

    df["SIMBAD_NAME"] = None
    df["SIMBAD_RA"] = None
    df["SIMBAD_DEC"] = None

    df["NED_NAME"] = None
    df["NED_RA"] = None
    df["NED_DEC"] = None

    df["SDSS_NAME"] = None
    df["SDSS_RA"] = None
    df["SDSS_DEC"] = None

    for i, row in df.iterrows():
        ra = row["RA"]
        dec = row["Dec"]

        # SIMBAD
        s_name, s_ra, s_dec = query_simbad_improved(ra, dec, radius_arcsec=simbad_radius)
        df.loc[i, ["SIMBAD_NAME", "SIMBAD_RA", "SIMBAD_DEC"]] = [s_name, s_ra, s_dec]

        # NED
        n_name, n_ra, n_dec = query_ned(ra, dec, radius_arcsec=ned_radius)
        df.loc[i, ["NED_NAME", "NED_RA", "NED_DEC"]] = [n_name, n_ra, n_dec]

        # SDSS
        sdss_name, sdss_ra, sdss_dec = query_sdss(ra, dec, radius_arcsec=sdss_radius)
        df.loc[i, ["SDSS_NAME", "SDSS_RA", "SDSS_DEC"]] = [sdss_name, sdss_ra, sdss_dec]

        print(f"Row {i}: SIMBAD={s_name}, NED={n_name}, SDSS={sdss_name}")

    return df



In [4]:
ndf = pd.read_csv('prg_36-for_public.csv',nrows=2)
ndf.head()


,Name_1,Name,a_host,b_host,PA_host,a_ring,b_ring,PA_ring,z,angle1,angle2,Dring/dhost,RA,Dec
0,SPRC3,3,5.7,2.8,325,12.1,2.9,66,0.037,86,74,2.11,10.264333,-9.941139
1,SPRC4,4,6.0,3.0,122,7.0,1.0,27,0.040,1,1,1.00,40.743300,-0.952600


In [ ]:
df = process_galaxy_catalog('prg_36-for_public.csv')


Row 0: SIMBAD=2MASX J00410343-0956284, NED=SPRC 003, SDSS=SDSS J10.26433-9.94113
Row 1: SIMBAD=SDSS J024258.42-005709.3, NED=SPRC 004, SDSS=SDSS J40.74346-0.95257
Row 2: SIMBAD=SDSS J075234.32+292049.7, NED=SDSS J075233.61+292051.1, SDSS=SDSS J118.14309+29.34716
Row 3: SIMBAD=LEDA 1489167, NED=None, SDSS=SDSS J125.15911+15.61660
Row 4: SIMBAD=Z 121-53, NED=SDSS J091815.89+202158.3, SDSS=SDSS J139.56654+20.36813
Row 5: SIMBAD=LEDA 1682548, NED=None, SDSS=SDSS J176.18342+23.16249
Row 6: SIMBAD=SPRC 39, NED=WISEA J130816.57+452238.9, SDSS=SDSS J197.07052+45.37644
Row 7: SIMBAD=LEDA 3567685, NED=None, SDSS=SDSS J236.85132+38.93066
Row 8: SIMBAD=Z 225-97, NED=None, SDSS=None
Row 9: SIMBAD=LEDA   65406, NED=None, SDSS=SDSS J312.02361+0.06885
Row 10: SIMBAD=MCG-01-07-035, NED=None, SDSS=None
